In [1]:

# 2022-Dez-21:
# PAREI AQUI;
# Usar os resultados de áreas dos picos e montar as curvas de eficiência

# 2022-Mar-16
# Fazendo ajuste polinomial nas eficiências:

In [2]:
import numpy as np
from numpy.polynomial import Polynomial as P  # 2020-09-06 Current recommended class
from scipy import linalg
import pandas as pd

In [3]:
En_CV_LoBg = np.array([81.00,
86.54,
121.78,
133.51,
302.85,
344.28,
356.02,
604.69,
661.66,
795.84,
964.13,
1085.91,
1112.12,
1173.24,
1332.50,
1408.01
])
# En_CV_GMX

In [4]:
Eff_CV_LoBg = np.array([4.4358,
4.4723,
5.4386,
5.5045,
3.1379,
2.8722,
2.7305,
1.7817,
1.6453,
1.4026,
1.1216,
1.0200,
0.9703,
0.8962,
0.8009,
0.7397 ])
# Eff_CV_GMX
Eff_CV_LoBg

array([4.4358, 4.4723, 5.4386, 5.5045, 3.1379, 2.8722, 2.7305, 1.7817,
       1.6453, 1.4026, 1.1216, 1.02  , 0.9703, 0.8962, 0.8009, 0.7397])

In [5]:
unc_perc = np.array([2.01,
1.43,
1.33,
2.11,
1.81,
1.42,
1.41,
2.24,
1.44,
3.68,
1.66,
1.71,
1.58,
0.87,
0.87,
1.51 ])
unc_perc

array([2.01, 1.43, 1.33, 2.11, 1.81, 1.42, 1.41, 2.24, 1.44, 3.68, 1.66,
       1.71, 1.58, 0.87, 0.87, 1.51])

In [6]:
ln_En_CV_LoBg = np.log (En_CV_LoBg)
ln_Eff_CV_LoBg = np.log (Eff_CV_LoBg)

In [7]:
N = 16

In [8]:
# 2022-Mar-23
# Parêntese para fazer nova calibração canal-energia do HPGe do projeto:

In [9]:
canal = np.array([210, 920, 1181, 7232])
ener = np.array([46.5, 186, 238.6, 1460.8])

In [10]:
projeto = P.fit(x=canal, y=ener, deg=1).convert()
projeto

Polynomial([1.76334373, 0.20170247], domain=[-1.,  1.], window=[-1.,  1.])

In [11]:
projeto_ener_canal = P.fit(x=ener, y=canal, deg=1).convert()
projeto_ener_canal

Polynomial([-8.72487544,  4.95776153], domain=[-1.,  1.], window=[-1.,  1.])

In [12]:
projeto_ener_canal(1173.2)

5807.720952746577

In [13]:
projeto_ener_canal(1332.486)

6597.422955976235

In [14]:
P.fit(x=ln_En_CV_LoBg, y=ln_Eff_CV_LoBg, deg=5).convert()

Polynomial([-3.72705775e+02,  3.00500400e+02, -9.55264764e+01,  1.50658279e+01,
       -1.18096853e+00,  3.67654984e-02], domain=[-1.,  1.], window=[-1.,  1.])

In [15]:
poly_w = P.fit(x=ln_En_CV_LoBg, y=ln_Eff_CV_LoBg, deg=5, w=1/unc_perc).convert()
poly_w

Polynomial([-4.52692893e+02,  3.72137803e+02, -1.21021144e+02,  1.95714003e+01,
       -1.57632172e+00,  5.05446265e-02], domain=[-1.,  1.], window=[-1.,  1.])

In [16]:
poly_w = P.fit(x=ln_En_CV_LoBg, y=ln_Eff_CV_LoBg, deg=5, w=1/unc_perc**2).convert()
poly_w

Polynomial([-5.90700260e+02,  4.94985076e+02, -1.64443537e+02,  2.71891133e+01,
       -2.23961917e+00,  7.34793811e-02], domain=[-1.,  1.], window=[-1.,  1.])

In [17]:
print(poly_w)

-590.7002599657337 + 494.98507553765165·x¹ - 164.44353702411448·x² +
27.18911329928743·x³ - 2.239619171069574·x⁴ + 0.07347938107242347·x⁵


In [18]:
poly_o = P.fit(x=ln_En_CV_LoBg, y=Eff_CV_LoBg, deg=5)
print(poly_o)

2.8983277899894047 - 3.6672204305624962·x¹ + 2.131490769929572·x² +
0.7157445482058078·x³ - 2.5321988401161253·x⁴ + 1.1796094492773173·x⁵


In [19]:
# 2023-Jun-14 O que será isso???
# weights = np.linspace(1, 2, N)
# Xw = X * np.sqrt(weights)[:, None]
# yw = y * np.sqrt(weights)
# print (linalg.lstsq(Xw, yw)[0])

In [20]:
weights = 1.0 / unc_perc**2.0
weights

array([0.24751863, 0.48902147, 0.56532308, 0.2246131 , 0.30524099,
       0.49593335, 0.50299281, 0.19929847, 0.48225309, 0.07384216,
       0.36289737, 0.34198557, 0.40057683, 1.32117849, 1.32117849,
       0.43857726])

In [21]:
Xw = ln_En_CV_LoBg * np.sqrt(weights)[:, None]
yw = ln_Eff_CV_LoBg * np.sqrt(weights)
print (linalg.lstsq(Xw, yw)[0])

[0.00337051 0.00342126 0.00368327 0.0037538  0.00438202 0.00448036
 0.00450608 0.00491238 0.00498143 0.00512306 0.00527019 0.00536142
 0.00537971 0.00542075 0.00551838 0.00556065]


In [22]:
print (linalg.lstsq(Xw, yw))

(array([0.00337051, 0.00342126, 0.00368327, 0.0037538 , 0.00438202,
       0.00448036, 0.00450608, 0.00491238, 0.00498143, 0.00512306,
       0.00527019, 0.00536142, 0.00537971, 0.00542075, 0.00551838,
       0.00556065]), array([], dtype=float64), 1, array([6.91304330e+01, 6.05929133e-15, 2.21553298e-15, 1.22736274e-15,
       9.44309798e-16, 8.59406481e-16, 6.73580396e-16, 6.08139955e-16,
       5.24981506e-16, 4.48117405e-16, 3.41109468e-16, 2.22575532e-16,
       2.00091774e-16, 1.41251642e-16, 3.40414085e-17, 1.99405749e-32]))


In [23]:
# 2023-Jun-14 Não sei o que é isso também...
# X = ln_En_CV_LoBg
# y = ln_Eff_CV_LoBg
# w1 = linalg.lstsq(X, y)
# print (w1)

In [24]:
np.log(2)

0.6931471805599453

In [25]:
# Now, we can insert the counting efficiency function:

In [26]:
# Curva de eficiência para frasco F100 do GMX (2021):
# Arquivo Effic_F100_GMX_2021.ods construído a partir de
# Nextcloud\Ipen\Radiometria\2021_Curvas-efic_Bessa\F100 GMX_2021.pdf

In [62]:
# 2023-Mar-21
# Lendo curva de eficiência
eff_F100_GMX_2021_df = pd.read_excel('../data/Effic_F100_GMX_2021.ods')
effics_df = eff_F100_GMX_2021_df
effics_df

,Engy,Effic,sEffic
0,59.54,6.4299,3.61
1,81.00,5.6528,3.61
2,276.40,1.6306,3.61
3,302.85,1.5006,3.61
4,356.02,1.4023,3.61
5,383.85,1.3216,3.61
6,604.69,0.9112,3.61
7,661.66,0.7900,3.61
8,795.84,0.7000,3.61
9,1155.00,0.5916,0.01


In [63]:
p_eff_log = P.fit(np.log(effics_df.Engy),
                  np.log(effics_df.Effic),
                  full=True,
                  w = 1/effics_df.sEffic, deg=5)
def p_eff(en):
    return np.exp(p_eff_log[0](np.log(en)))

In [64]:
p_eff_log

(Polynomial([ 0.51104917, -1.54282532,  0.42082601,  0.36310567, -0.3132035 ,
        -0.06636575], domain=[4.08664836, 7.19481216], window=[-1.,  1.]),
 [array([0.00174484]),
  6,
  array([2.44946008e+00, 1.04967799e-02, 5.50802223e-03, 2.01458065e-03,
         8.07092181e-04, 2.92462463e-04]),
  2.6645352591003757e-15])

In [65]:
p_eff(effics_df.Engy)

0     6.454368
1     5.607742
2     1.698686
3     1.552736
4     1.335262
5     1.249593
6     0.884475
7     0.834158
8     0.744488
9     0.591599
10    0.585366
11    0.533971
Name: Engy, dtype: float64

In [66]:
log_unc_fitted_effic = P([4.70033902423E+01, -2.82179877310E+01, 5.72952770082E+00, -3.88458849293E-01])
log_unc_fitted_effic

Polynomial([ 47.00339024, -28.21798773,   5.7295277 ,  -0.38845885], domain=[-1,  1], window=[-1,  1])

In [67]:
# Incerteza percentual da curva F100_GMX_2021 para os valores de eficiência
# Os coeficientes foram tirados do InterWinner e a curva reconstruída aqui.
unc_perc_fitted_effic = np.exp(log_unc_fitted_effic(np.log(effics_df.Engy)))
unc_perc_fitted_effic

0     2.366059
1     1.972769
2     1.533736
3     1.437744
4     1.233116
5     1.126118
6     0.470105
7     0.365696
8     0.198657
9     0.037309
10    0.034281
11    0.016454
Name: Engy, dtype: float64

In [68]:
effics_df

,Engy,Effic,sEffic
0,59.54,6.4299,3.61
1,81.00,5.6528,3.61
2,276.40,1.6306,3.61
3,302.85,1.5006,3.61
4,356.02,1.4023,3.61
5,383.85,1.3216,3.61
6,604.69,0.9112,3.61
7,661.66,0.7900,3.61
8,795.84,0.7000,3.61
9,1155.00,0.5916,0.01


In [70]:
effics_df["effic_func"] = 743 # p_eff_log
effics_df

,Engy,Effic,sEffic,effic_func
0,59.54,6.4299,3.61,743
1,81.00,5.6528,3.61,743
2,276.40,1.6306,3.61,743
3,302.85,1.5006,3.61,743
4,356.02,1.4023,3.61,743
5,383.85,1.3216,3.61,743
6,604.69,0.9112,3.61,743
7,661.66,0.7900,3.61,743
8,795.84,0.7000,3.61,743
9,1155.00,0.5916,0.01,743


In [81]:
effics_df.loc[(5, 'effic_func')] = p_eff_log
effics_df

/home/marcelofm/ogrsp_in_github/venv/lib/python3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


ValueError: Must have equal len keys and value when setting with an iterable

In [82]:
effics_df.effic_func[2] = p_eff_log
effics_df

,Engy,Effic,sEffic,effic_func
0,59.54,6.4299,3.61,743
1,81.00,5.6528,3.61,743
2,276.40,1.6306,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
3,302.85,1.5006,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
4,356.02,1.4023,3.61,743
5,383.85,1.3216,3.61,743
6,604.69,0.9112,3.61,743
7,661.66,0.7900,3.61,743
8,795.84,0.7000,3.61,743
9,1155.00,0.5916,0.01,743


In [83]:
effics_df.effic_func[3] = p_eff_log
effics_df

,Engy,Effic,sEffic,effic_func
0,59.54,6.4299,3.61,743
1,81.00,5.6528,3.61,743
2,276.40,1.6306,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
3,302.85,1.5006,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
4,356.02,1.4023,3.61,743
5,383.85,1.3216,3.61,743
6,604.69,0.9112,3.61,743
7,661.66,0.7900,3.61,743
8,795.84,0.7000,3.61,743
9,1155.00,0.5916,0.01,743


In [84]:
effics_df.loc[1,('effic_func')]

743

In [85]:
pkl_name = 'f100_gmx_2021.pkl'

In [86]:
effics_df.to_pickle(pkl_name)

In [87]:
calib_pkl_name = 'f100_gmx_2021.pkl'
calib_df = pd.read_pickle(calib_pkl_name)
calib_df

,Engy,Effic,sEffic,effic_func
0,59.54,6.4299,3.61,743
1,81.00,5.6528,3.61,743
2,276.40,1.6306,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
3,302.85,1.5006,3.61,"((0.5110491663154504, -1.542825320223859, 0.42..."
4,356.02,1.4023,3.61,743
5,383.85,1.3216,3.61,743
6,604.69,0.9112,3.61,743
7,661.66,0.7900,3.61,743
8,795.84,0.7000,3.61,743
9,1155.00,0.5916,0.01,743
